In [1]:
'''import all necessary libraries'''

import cx_Oracle #import the oracle database libraries
import pandas as pd #connect to the pandas libraries
import numpy as np #import numpy
import getpass #import the password libraries in order to keep database password concealed

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import sys
print (sys.version)
pd.options.display.float_format = '{:0.6f}'.format

3.6.5 |Anaconda custom (64-bit)| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]


In [2]:
'''connect to the oracle server used to pull down the 
currency and misuse of authorization information'''

username=input("please enter your username")
password=getpass.getpass("Enter your password: ")

con=cx_Oracle.connect(username,password,'ODSPEU1.ams') #enter database details
print (con.version) # prnt the conversion details

please enter your username djohnson
Enter your password:  ·············


12.1.0.2.0


In [ ]:
df=pd.read_sql(
''' 
select extract(month from CAST(ev.MONTH AS DATE)) AS MONTH,EV.CONTRACT_ID,EV.MID,EV.MID_NAME
,EV.CREDIT_DEBIT_CARD,EV.REGION,EV.CARD_PLAN,EV.CARD_TYPE,EV.CHARGE_TYPE_DESCRIPTION,EV.PRODUCT_CODE
,SUM(EV.GROSS_VOLUME) AS VOLUME,SUM(EV.GROSS_FLOW_EUR) AS FLOW_EUR,SUM(EV.SCHEME_FEES_EUR) AS SF
,EV.TRANS_CURRENCY,EV.PROCESSING_FEE_CURRENCY,EV.VAT_CURRENCY,EV.ASSESSMENT_CURRENCY
from FDWO.ACQUIRER_ELAVON_EUR_COST ev
where EV.MONTH >=to_date('4/1/2018','MM/DD/YYYY')
and GROSS_VOLUME <>0
group by EV.MID_NAME,
EV.MID_NAME,EV.CREDIT_DEBIT_CARD,EV.REGION,EV.CARD_PLAN,EV.CARD_TYPE,EV.CHARGE_TYPE_DESCRIPTION,
EV.PRODUCT_CODE,EV.TRANS_CURRENCY,EV.PROCESSING_FEE_CURRENCY
,EV.VAT_CURRENCY,EV.ASSESSMENT_CURRENCY,EV.CONTRACT_ID,EV.MID,EV.MONTH
''', con)

In [4]:
'''client names'''
df_cn=pd.read_sql(
''' 
select MM.CONTRACT_ID, MM.CLIENT_NAME,COUNTRY AS MERCHANT_COUNTRY
from FDWO.MB_MERCHANTS mm
group by MM.CONTRACT_ID, MM.CLIENT_NAME,COUNTRY
''', con)

OperationalError: ORA-03113: end-of-file on communication channel
Process ID: 104503
Session ID: 1001 Serial number: 38058

In [ ]:
'''ic_plus_type'''
df_ic=pd.read_sql(
'''
select CC.CONTRACT_ID, CC.PRODUCT_CODE,CC.TRANSACTION_CURRENCY,CC.ICPLUS_TYPE
from FDWO.CC_PROCESSING_DATA_PT cc
where CC.TRANSACTION_TYPE='TRANSACTION'
and CC.PAYMENT_PROCESSOR='Elavon (Europe)'
and CC.INVOICE_CURRENCY is not null
and CC.MATCH_DATE between to_date('4/1/2018','MM/DD/YYYY') and to_date('8/31/2018','MM/DD/YYYY')
group by CC.CONTRACT_ID, CC.PRODUCT_CODE, CC.TRANSACTION_CURRENCY,cc.ICPLUS_TYPE
''', con)

In [3]:
elavon_cntry=pd.read_csv('G:\Pricing\Dale\Elavon_Country_Identification\elavon_merchant_countries.csv',dtype={'CONTRACT_ID':'str'}) #import the elavon country list
iep=pd.read_csv('d:\\djohnson\\Desktop\\aib_theoreticals_desktop_ref\\aib_iep_theoreticals_V3.csv',encoding='ISO-8859-1') #import Ahron's data
df_mc=pd.read_csv('d:\\djohnson\\Desktop\\aib_theoreticals_desktop_ref\\multicurrency.csv',encoding='ISO-8859-1') #import multicurrency designation

In [ ]:
iep=iep[['Concatenate','Security Level','Fixed','Percent','PERIOD']]
iep['Percent']=iep['Percent'].astype(float)

In [ ]:
df1=df.copy()
df_cn_1=df_cn.copy()
df_ic_1=df_ic.copy()
elavon_cntry_1=elavon_cntry.copy()

In [ ]:
master_data_set=pd.merge(df1,df_cn_1,how='left',on=['CONTRACT_ID'])
master_data_set=pd.merge(master_data_set,elavon_cntry,how='left',on=['CHARGE_TYPE_DESCRIPTION'])

In [ ]:
master_data_set.head()

In [ ]:
for x in range(0,len(master_data_set['MONTH'])):
    if master_data_set.loc[x,'COUNTRY'] is None:
         master_data_set.loc[x,'MERCHANT_COUNTRY']=master_data_set.loc[x,'COUNTRY']

In [ ]:
#master_data_set=master_data_set.rename(columns={'COUNTRY':'MERCHANT_COUNTRY'})
master_data_set=master_data_set.rename(columns={'TRANS_CURRENCY':'TRANSACTION_CURRENCY'})
master_data_set=master_data_set.rename(columns={'SOURCE':'MERCHANT_COUNTRY_INFO_SOURCE'})

In [ ]:
master_data_set['MERCHANT_COUNTRY_UPDATED_FROM:']=''
master_data_set['MERCHANT_COUNTRY_UPDATED_FROM:'][(master_data_set['MERCHANT_COUNTRY']=='Gibraltar')]='Gibraltar'
master_data_set['MERCHANT_COUNTRY_UPDATED_FROM:'][(master_data_set['MERCHANT_COUNTRY']=='Andorra')]='Andorra'
master_data_set['MERCHANT_COUNTRY_UPDATED_FROM:'][(master_data_set['MERCHANT_COUNTRY']=='Panama')]='Panama'
master_data_set['MERCHANT_COUNTRY_UPDATED_FROM:'][(master_data_set['MERCHANT_COUNTRY']=='Monaco')]='Monaco'
master_data_set['MERCHANT_COUNTRY_UPDATED_FROM:'][(master_data_set['MERCHANT_COUNTRY']=='Australia')]='Australia'
master_data_set['MERCHANT_COUNTRY_UPDATED_FROM:'][(master_data_set['MERCHANT_COUNTRY']=='New Zealand')]='New Zealand'
master_data_set['MERCHANT_COUNTRY_UPDATED_FROM:'][(master_data_set['MERCHANT_COUNTRY']=='Singapore')]='Singapore'
master_data_set['MERCHANT_COUNTRY_UPDATED_FROM:'][(master_data_set['MERCHANT_COUNTRY']=='Thailand')]='Thailand'
master_data_set['MERCHANT_COUNTRY_UPDATED_FROM:'][(master_data_set['MERCHANT_COUNTRY']=='Malaysia')]='Malaysia'
master_data_set['MERCHANT_COUNTRY_UPDATED_FROM:'][(master_data_set['MERCHANT_COUNTRY']=='Japan')]='Japan'
master_data_set['MERCHANT_COUNTRY_UPDATED_FROM:'][(master_data_set['MERCHANT_COUNTRY']=='Peru')]='Peru'
master_data_set['MERCHANT_COUNTRY_UPDATED_FROM:'][(master_data_set['MERCHANT_COUNTRY']=='Chile')]='Chile'
master_data_set['MERCHANT_COUNTRY_UPDATED_FROM:'][(master_data_set['MERCHANT_COUNTRY']=='Colombia')]='Colombia'
master_data_set['MERCHANT_COUNTRY_UPDATED_FROM:'][(master_data_set['MERCHANT_COUNTRY']=='Azerbaijan')]='Azerbaijan'
master_data_set['MERCHANT_COUNTRY_UPDATED_FROM:'][(master_data_set['MERCHANT_COUNTRY']=='China')]='China'
master_data_set['MERCHANT_COUNTRY_UPDATED_FROM:'][(master_data_set['MERCHANT_COUNTRY']=='Brazil')]='Brazil'
master_data_set['MERCHANT_COUNTRY_UPDATED_FROM:'][(master_data_set['MERCHANT_COUNTRY']=='Ecuador')]='Ecuador'
master_data_set['MERCHANT_COUNTRY_UPDATED_FROM:'][(master_data_set['MERCHANT_COUNTRY']=='United States')]='United States'

master_data_set['MERCHANT_COUNTRY'][(master_data_set['MERCHANT_COUNTRY']=='Gibraltar')]='UK'
master_data_set['MERCHANT_COUNTRY'][(master_data_set['MERCHANT_COUNTRY']=='Andorra')]='France'
master_data_set['MERCHANT_COUNTRY'][(master_data_set['MERCHANT_COUNTRY']=='Panama')]='UK'
master_data_set['MERCHANT_COUNTRY'][(master_data_set['MERCHANT_COUNTRY']=='Monaco')]='France'
master_data_set['MERCHANT_COUNTRY'][(master_data_set['MERCHANT_COUNTRY']=='Australia')]='Ireland'
master_data_set['MERCHANT_COUNTRY'][(master_data_set['MERCHANT_COUNTRY']=='New Zealand')]='UK'
master_data_set['MERCHANT_COUNTRY'][(master_data_set['MERCHANT_COUNTRY']=='Singapore')]='UK'
master_data_set['MERCHANT_COUNTRY'][(master_data_set['MERCHANT_COUNTRY']=='Thailand')]='UK'
master_data_set['MERCHANT_COUNTRY'][(master_data_set['MERCHANT_COUNTRY']=='Malaysia')]='UK'
master_data_set['MERCHANT_COUNTRY'][(master_data_set['MERCHANT_COUNTRY']=='Japan')]='UK'
master_data_set['MERCHANT_COUNTRY'][(master_data_set['MERCHANT_COUNTRY']=='Peru')]='UK'
master_data_set['MERCHANT_COUNTRY'][(master_data_set['MERCHANT_COUNTRY']=='Chile')]='UK'
master_data_set['MERCHANT_COUNTRY'][(master_data_set['MERCHANT_COUNTRY']=='Colombia')]='UK'
master_data_set['MERCHANT_COUNTRY'][(master_data_set['MERCHANT_COUNTRY']=='Azerbaijan')]='UK'
master_data_set['MERCHANT_COUNTRY'][(master_data_set['MERCHANT_COUNTRY']=='China')]='UK'
master_data_set['MERCHANT_COUNTRY'][(master_data_set['MERCHANT_COUNTRY']=='Brazil')]='UK'
master_data_set['MERCHANT_COUNTRY'][(master_data_set['MERCHANT_COUNTRY']=='Ecuador')]='UK'
master_data_set['MERCHANT_COUNTRY'][(master_data_set['MERCHANT_COUNTRY']=='United States')]='UK'

In [ ]:
master_data_set=pd.merge(master_data_set,df_mc,on=['MERCHANT_COUNTRY'],how='left').copy() # identify if it's a multicurrency country

In [ ]:
master_data_set['BRAND']=''
for x in range(0,len(master_data_set['MONTH'])):
    if "MC " in master_data_set.loc[x,'CHARGE_TYPE_DESCRIPTION']:
        master_data_set.loc[x,'BRAND']='MasterCard'
    elif "VI " in master_data_set.loc[x,'CHARGE_TYPE_DESCRIPTION']:
        master_data_set.loc[x,'BRAND']='Visa'
    elif "DC " in master_data_set.loc[x,'CHARGE_TYPE_DESCRIPTION']:
        master_data_set.loc[x,'BRAND']='Discover'

In [ ]:
master_data_set['CARD_TYPES']=''
for x in range(0,len(master_data_set['MONTH'])):
    if "D" in master_data_set.loc[x,'CREDIT_DEBIT_CARD']:
        master_data_set.loc[x,'CARD_TYPES']='Debit'
    elif "C" in master_data_set.loc[x,'CREDIT_DEBIT_CARD']:
        master_data_set.loc[x,'CARD_TYPES']=''

In [ ]:
'''build Regionality for Concatenation'''
master_data_set['Regionality']=''
for x in range(0,len(master_data_set['MONTH'])):
    if master_data_set.loc[x,'REGION']=='INTER':
        master_data_set.loc[x,'Regionality']='Inter-Region'
    elif master_data_set.loc[x,'REGION']=='':
        master_data_set.loc[x,'Regionality']='Inter-Region'        
    elif master_data_set.loc[x,'REGION']=='INTRA':
        master_data_set.loc[x,'Regionality']='Intra-Region'
    elif master_data_set.loc[x,'REGION']=='DOMESTIC':
        master_data_set.loc[x,'Regionality']='Domestic'
    else:
        master_data_set.loc[x,'Regionality']='Inter-Region'        

In [ ]:
'''build period of time which will be used to determine which fees will be applied'''
master_data_set['PERIOD']=''
for x in range(0,len(master_data_set['MONTH'])):
    if master_data_set.loc[x,'MONTH']<=7:
        master_data_set.loc[x,'PERIOD']='prejuly18'
    elif master_data_set.loc[x,'MONTH']>7:
        master_data_set.loc[x,'PERIOD']='postjuly18'

In [ ]:
master_data_set['Security Level']='Nonsecure'

In [ ]:
master_data_set['Product']=master_data_set['BRAND']+master_data_set['CARD_TYPES']
master_data_set['Concatenate']=master_data_set['Product']+master_data_set['MERCHANT_COUNTRY']+master_data_set['Regionality']

In [ ]:
df_ic_1['CONTRACT_ID']=df_ic_1['CONTRACT_ID'].astype(str)

In [ ]:
master_data_set=pd.merge(master_data_set,df_ic_1,how='left',on=['CONTRACT_ID','TRANSACTION_CURRENCY','PRODUCT_CODE']).copy()

In [ ]:
master_data_set=pd.merge(master_data_set,iep,on=['Security Level','Concatenate','PERIOD'],how='left').copy()

In [ ]:
master_data_set['MULTI_CURRENCY_FEE']=0

master_data_set['MULTI_CURRENCY_FEE'][(master_data_set['MULTI_CURRENCY_APPLICABLE']=='Yes')&(master_data_set['BRAND']=='MasterCard')&(master_data_set['Regionality']=='Domestic')&(master_data_set['TRANSACTION_CURRENCY']!=master_data_set['ASSESSMENT_CURRENCY'])]=.0003
master_data_set['MULTI_CURRENCY_FEE'][(master_data_set['MULTI_CURRENCY_APPLICABLE']=='Yes')&(master_data_set['BRAND']=='Maestro')&(master_data_set['Regionality']=='Domestic')&(master_data_set['TRANSACTION_CURRENCY']!=master_data_set['ASSESSMENT_CURRENCY'])]=.0003

master_data_set['MULTI_CURRENCY_FEE'][(master_data_set['MULTI_CURRENCY_APPLICABLE']=='Yes')&(master_data_set['BRAND']=='MasterCard')&(master_data_set['Regionality']=='Intra-Region')&(master_data_set['TRANSACTION_CURRENCY']!=master_data_set['ASSESSMENT_CURRENCY'])]=.0003
master_data_set['MULTI_CURRENCY_FEE'][(master_data_set['MULTI_CURRENCY_APPLICABLE']=='Yes')&(master_data_set['BRAND']=='Maestro')&(master_data_set['Regionality']=='Intra-Region')&(master_data_set['TRANSACTION_CURRENCY']!=master_data_set['ASSESSMENT_CURRENCY'])]=.0003

master_data_set['MULTI_CURRENCY_FEE'][(master_data_set['MULTI_CURRENCY_APPLICABLE']=='Yes')&(master_data_set['BRAND']=='MasterCard')&(master_data_set['Regionality']=='Inter-Region')&(master_data_set['TRANSACTION_CURRENCY']!=master_data_set['ASSESSMENT_CURRENCY'])]=.0013
master_data_set['MULTI_CURRENCY_FEE'][(master_data_set['MULTI_CURRENCY_APPLICABLE']=='Yes')&(master_data_set['BRAND']=='Maestro')&(master_data_set['Regionality']=='Inter-Region')&(master_data_set['TRANSACTION_CURRENCY']!=master_data_set['ASSESSMENT_CURRENCY'])]=.0013


master_data_set['HYPOTHETICAL_SF']=(master_data_set['Fixed']*master_data_set['VOLUME'])+(master_data_set['Percent']*master_data_set['FLOW_EUR'])+(master_data_set['MULTI_CURRENCY_FEE']*master_data_set['FLOW_EUR'])

In [ ]:
master_data_set['MULTI_CURRENCY_APPLICABLE'][(master_data_set['MULTI_CURRENCY_FEE']==0)]='No'

In [ ]:
master_data_set=master_data_set[(master_data_set['BRAND']!='Discover')].copy()

In [ ]:
master_data_set.head()

In [ ]:
master_data_set=master_data_set.drop_duplicates().copy()

In [ ]:
master_data_set.to_csv('d:\\djohnson\\Desktop\\elavon_sf_analysis\\elavon_sf_analysis.csv')